In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Helpful
import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
from src.models.EMOS_global.EMOS_global_load_score import *
from src.models.EMOS_local.EMOS_local_load_score import *
from src.models.EMOS_global.EMOS_global_load_model import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
import data.processed.load_data_processed_denormed as ldpd
from src.models.CRPS_baseline.CRPS_load import *
from src.models.EMOS_local.EMOS_local_train import *

2023-05-31 19:55:12.697367: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-31 19:55:12.697388: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### 1. Load Raw Data

In [3]:
EMOS_local_t2m_scores = EMOS_local_load_score("t2m")
EMOS_global_t2m_scores = EMOS_global_load_score("t2m")

<IPython.core.display.Javascript object>

In [4]:
len(EMOS_local_t2m_scores)

31

<IPython.core.display.Javascript object>

In [5]:
EMOS_local_t2m_scores[0].shape

(120, 130)

<IPython.core.display.Javascript object>

In [7]:
EMOS_local_t2m_scores[30]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

<IPython.core.display.Javascript object>

In [6]:
for i in range(len(EMOS_local_t2m_scores)):
    print(np.mean(EMOS_local_t2m_scores[i]))

328.4947446075649
330.23082720595545
327.9483725371961
327.4246856143493
325.12237542186887
329.68265990216054
328.4815737422798
327.80967964218854
326.97397396183334
332.5469142508466
327.98502607177977
328.61883506726286
326.3603014033997
327.59675320344036
329.88643152674564
328.0712323588493
325.48345232737216
330.7377808812805
328.6731864870094
324.91899653014787
323.73391273756556
329.8773844068397
331.857066076039
326.8387646587133
323.4507281165549
324.2752901651796
324.22444877192015
328.4363450070129
324.48297908063785
330.94390141942785
0.0


<IPython.core.display.Javascript object>

In [8]:
for i in range(len(EMOS_global_t2m_scores)):
    print(np.median(EMOS_global_t2m_scores[i]))

0.3579158720675525
0.4294854308486648
0.43580610345285276
0.46634451031549323
0.4805410959290852
0.5147794461997237
0.5319366754911081
0.5655238974572119
0.5731764403980474
0.5984597704946297
0.6204348943894081
0.6523478367241466
0.6587426287984816
0.6819800091642969
0.7016747915923386
0.7306790653307432
0.7344800723014707
0.7602097936026542
0.7728042089225389
0.8031066091325454
0.8045481476610289
0.838901271578145
0.8489514579129032
0.8728372701922087
0.8741058417649076
0.8986773938169144
0.9177029102092923
0.9431298210736898
0.9401560790901858
0.969711973380655
0.980268295618246


<IPython.core.display.Javascript object>

### 2. Hyperparameter tuning:

In [9]:
var_num = 2
lead_time = 0
batch_size = 128
epochs = 10
lr = 0.01
validation_split = 0.2
optimizer = "Adam"

<IPython.core.display.Javascript object>

In [ ]:
EMOS_local_train(
    var_num,
    lead_time,
    batch_size=batch_size,
    epochs=epochs,
    lr=lr,
    validation_split=validation_split,
    optimizer=optimizer,
    save=False,
)

Epoch 1/10
9/9 [==============================] - 0s 14ms/step - loss: 83.8699 - val_loss: 69.7883
Epoch 2/10
9/9 [==============================] - 0s 4ms/step - loss: 58.3929 - val_loss: 44.2366
Epoch 3/10
9/9 [==============================] - 0s 4ms/step - loss: 32.9202 - val_loss: 18.6839
Epoch 4/10
9/9 [==============================] - 0s 4ms/step - loss: 8.2230 - val_loss: 3.1666
Epoch 5/10
9/9 [==============================] - 0s 4ms/step - loss: 5.8497 - val_loss: 4.5275
Epoch 6/10
9/9 [==============================] - 0s 5ms/step - loss: 1.9953 - val_loss: 2.4351
Epoch 7/10
9/9 [==============================] - 0s 3ms/step - loss: 1.5582 - val_loss: 0.9555
Epoch 8/10
9/9 [==============================] - 0s 3ms/step - loss: 1.0444 - val_loss: 0.6923
Epoch 9/10
9/9 [==============================] - 0s 4ms/step - loss: 0.8047 - val_loss: 0.6807
Epoch 10/10
9/9 [==============================] - 0s 3ms/step - loss: 0.7168 - val_loss: 0.6684
Epoch 1/10
9/9 [================

### 3. Trying to load models: